In [ ]:
from bokeh.models import CustomJS, ColumnDataSource
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.plotting import figure

import param
import pandas as pd
from pandas import json_normalize

import altair as alt
import numpy as np
import panel as pn

pn.extension("vega")

In [ ]:
data = pd.DataFrame({"x": np.arange(0, 12, 1),
              "y": np.arange(0, 1.2, 0.1),
             "day": np.tile([1,2,3,4], 3),
             "user": np.repeat(["a","b","c"], 4)})

class UserDashboard(param.Parameterized):

    user_id = param.String(default="")       

    @param.depends('user_id')
    def view(self):
        if self.user_id in data.user.to_list():
            return alt.Chart(data = data.query(f"user=='{self.user_id}'"),
                             title=f"Y over time for {','.join(self.user_id)}",
                             width=400, height=200
                            ).mark_bar().encode(x="day", y="y")
        else: 
            return f"User {self.user_id} not found"

user = UserDashboard()
#pn.Column(user, user.view)

In [ ]:
class GroupDashboard(param.Parameterized):

    user_ids = param.List(["a","b"])       

    @param.depends('user_ids')
    def view(self):
        if len(self.user_ids) > 0:
            return alt.Chart(data = data.loc[data.user.isin(self.user_ids)], 
                             title=f"Y over time for {','.join(self.user_ids)}",
                             width=400, height=200
                            ).mark_point().encode(x="day", y="y", color="user")
        else: 
            return None

group = GroupDashboard()
pn.Column(group, group.view)

In [ ]:
# Create a Scatter Chart
selector = ColumnDataSource(data=data.to_dict(orient="list"))
scatter = figure(plot_width=400, plot_height=400, tools="lasso_select", title="Select Here")
scatter.circle('x', 'y', source=selector)

# Create Table
table_data = ColumnDataSource(data=data.sample(0).to_dict(orient="list"))
table = DataTable(
    columns=[TableColumn(field=x, title=x) for x in data.columns],
    source=table_data, 
    height=400, 
    width=400, 
    autosize_mode="fit_columns"
)

# Create the filter-functionality Scatter -> Table
js_mappings = {
            "selector": selector, 
            "table_data": table_data,
            "data_source": eval(data.to_json(orient="records"))
        }
selector.selected.js_on_change('indices', CustomJS(args=js_mappings, code="""
        var inds = cb_obj.indices;
        var selector = selector.data;
        var table = table_data.data;
        console.log()

        Object.keys(table).forEach(function(key){
            console.log(key)
            table[key] = []
            for (var i = 0; i < inds.length; i++) {
                table[key].push(data_source[inds[i]][key])
            }
        })
        table_data.change.emit();
    """))
        

# Create the filter-functionality Scatter -> Panel Group Dashboard
def select_group(attr, old, new):
    group.user_ids = data.loc[new,"user"].drop_duplicates().to_list()
    
selector.selected.on_change("indices", select_group)

# Create the filter-functionality Table -> Panel User Dashboard
def select_user(attr, old, new):
    user.user_id = data.loc[selector.selected.indices[new[0]],"user"]

table_data.selected.on_change("indices", select_user)

# Show the whole Dashboard
pn.Column(pn.Row(scatter, table), 
          pn.Row(group.view, user.view))